In [10]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit
import numpy as np
from numpy.linalg import norm,inv,matrix_rank
import h5py

from Code.SGDPLS import SGDPLS
from Code.OLPLS import OLPLS
from Code.CIPLS import CIPLS
from Code.SIMPLS import SIMPLS
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS
from Code.IPLS import IPLS
# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc
tictoc=TicToc()

In [3]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)
    
    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

In [4]:
tmp = h5py.File('./data/TW_PM25.h5', 'r')
X_train, Y_train = tmp['X_train'], tmp['Y_train']
n_train = X_train.shape[0]

X_train = X_train[0:n_train]
Y_train = Y_train[0:n_train]
tmp.close()



In [8]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=17, test_size=30)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [14]:
n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLSRegression(), parameters,
                   cv=tscv, n_jobs=4,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=PLSRegression(), n_jobs=4,
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15]},
             scoring='neg_root_mean_squared_error')


CV elapsed time: 45.60s
best parameter:  {'n_components': 10} ; score:  -7.1954502e+00


In [30]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,11))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, n_jobs=4, verbose=3,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SIMPLS(), n_jobs=4,
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='neg_root_mean_squared_error', verbose=3)
Fitting 17 folds for each of 10 candidates, totalling 170 fits


CV elapsed time: 65.19s
best parameter:  {'n_components': 8} ; score:  -8.4602085e+00


[CV 1/17] END ..................n_components=1;, score=-9.513 total time=   0.5s
[CV 5/17] END ..................n_components=1;, score=-7.509 total time=   0.6s
[CV 11/17] END .................n_components=1;, score=-8.250 total time=   0.7s
[CV 15/17] END .................n_components=1;, score=-8.541 total time=   0.7s
[CV 2/17] END ..................n_components=2;, score=-8.559 total time=   0.7s
[CV 6/17] END ..................n_components=2;, score=-7.560 total time=   0.8s
[CV 10/17] END ................n_components=2;, score=-11.760 total time=   0.8s
[CV 14/17] END .................n_components=2;, score=-8.952 total time=   0.9s
[CV 1/17] END ..................n_components=3;, score=-9.452 total time=   0.8s
[CV 6/17] END ..................n_components=3;, score=-7.966 total time=   0.9s
[CV 8/17] END ..................n_components=3;, score=-9.989 total time=   1.0s
[CV 13/17] END .................n_components=3;, score=-9.689 total time=   1.2s
[CV 17/17] END .............

[CV 3/17] END ..................n_components=1;, score=-8.659 total time=   0.5s
[CV 6/17] END ..................n_components=1;, score=-7.751 total time=   0.5s
[CV 9/17] END ..................n_components=1;, score=-8.676 total time=   0.6s
[CV 13/17] END .................n_components=1;, score=-9.648 total time=   0.6s
[CV 17/17] END .................n_components=1;, score=-7.370 total time=   0.7s
[CV 5/17] END ..................n_components=2;, score=-7.553 total time=   0.7s
[CV 9/17] END ..................n_components=2;, score=-8.525 total time=   0.9s
[CV 13/17] END .................n_components=2;, score=-9.663 total time=   0.8s
[CV 17/17] END .................n_components=2;, score=-7.137 total time=   0.9s
[CV 4/17] END ..................n_components=3;, score=-6.806 total time=   0.8s
[CV 7/17] END ..................n_components=3;, score=-9.619 total time=   0.9s
[CV 11/17] END .................n_components=3;, score=-8.052 total time=   1.1s
[CV 16/17] END .............

[CV 2/17] END ..................n_components=1;, score=-8.656 total time=   0.5s
[CV 8/17] END .................n_components=1;, score=-10.345 total time=   0.6s
[CV 12/17] END .................n_components=1;, score=-7.468 total time=   0.7s
[CV 16/17] END .................n_components=1;, score=-7.357 total time=   0.8s
[CV 3/17] END ..................n_components=2;, score=-8.578 total time=   0.7s
[CV 7/17] END ..................n_components=2;, score=-9.462 total time=   0.7s
[CV 11/17] END .................n_components=2;, score=-7.947 total time=   0.7s
[CV 15/17] END .................n_components=2;, score=-8.543 total time=   0.9s
[CV 2/17] END ..................n_components=3;, score=-8.420 total time=   0.8s
[CV 5/17] END ..................n_components=3;, score=-7.802 total time=   1.1s
[CV 10/17] END ................n_components=3;, score=-11.716 total time=   1.0s
[CV 14/17] END .................n_components=3;, score=-8.918 total time=   1.1s
[CV 1/17] END ..............

[CV 4/17] END ..................n_components=1;, score=-6.948 total time=   0.5s
[CV 7/17] END ..................n_components=1;, score=-9.463 total time=   0.5s
[CV 10/17] END ................n_components=1;, score=-12.187 total time=   0.6s
[CV 14/17] END .................n_components=1;, score=-9.083 total time=   0.7s
[CV 1/17] END ..................n_components=2;, score=-9.484 total time=   0.6s
[CV 4/17] END ..................n_components=2;, score=-6.945 total time=   0.7s
[CV 8/17] END .................n_components=2;, score=-10.032 total time=   0.7s
[CV 12/17] END .................n_components=2;, score=-7.439 total time=   0.8s
[CV 16/17] END .................n_components=2;, score=-6.881 total time=   1.0s
[CV 3/17] END ..................n_components=3;, score=-8.346 total time=   1.1s
[CV 9/17] END ..................n_components=3;, score=-8.546 total time=   0.9s
[CV 12/17] END .................n_components=3;, score=-7.449 total time=   0.9s
[CV 15/17] END .............

In [ ]:
import Code.SGDPLS
importlib.reload(Code.SGDPLS)
from Code.SGDPLS import SGDPLS 
parameters = {'epochs': [100], 'eta':[1e-2]}
# parameters = {'epochs': [100, 200, 300], 'eta':[1e-2, 1e-3, 1e-4]}
clf = GridSearchCV(SGDPLS(n_componets=10),  parameters,
                   cv=tscv, n_jobs=2,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))

In [8]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

parameters = {'mu': [1e-4, 1e-5, 1e-6], 'amnesic': [1e-2] + list(np.arange(0.1,1.0,0.1)) + [0.99]}
clf = GridSearchCV(OLPLS(n_components=10), parameters,
                   cv=tscv, n_jobs=4,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=OLPLS(), n_jobs=4,
             param_grid={'amnesic': [0.01, 0.1, 0.2, 0.30000000000000004, 0.4,
                                     0.5, 0.6, 0.7000000000000001, 0.8, 0.9,
                                     0.99],
                         'mu': [0.0001, 1e-05, 1e-06]},
             scoring='neg_root_mean_squared_error')


/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:151: RuntimeWarning: invalid value encountered in divide
  b = b / (np.dot(wtemp.T, wtemp))
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:151: RuntimeWarning: invalid value encountered in divide
  b = b / (np.dot(wtemp.T, wtemp))
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:151: RuntimeWarning: invalid value encountered in divide
  b = b / (np.dot(wtemp.T, wtemp))
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:151: RuntimeWarning: invalid value encountered in divide
  b = b / (np.dot(wtemp.T, wtemp))


/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: invalid value encountered in divide
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: invalid value encountered in divide
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: invalid value encountered in divide
  a = a / (np.dot(wtemp, wtemp) ** 2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:148: RuntimeWarning: overflow encountered in scalar power
  a = a / (n

/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:124: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp)**2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:124: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp)**2)


/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:124: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp)**2)
/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py:124: RuntimeWarning: overflow encountered in scalar power
  a = a / (np.dot(wtemp, wtemp)**2)


...

/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py", line 1

/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py", line 1

/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/video/data/tingan/GitRep/IPLSreview/Code/OLPLS.py", line 1

/home/tingan/miniconda3/envs/pls/lib/python3.10/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [         nan          nan -15.59388028          nan          nan
 -14.86262476          nan          nan -14.94785698          nan
          nan -15.01935609          nan          nan -15.48446353
          nan          nan -15.68091893          nan          nan
 -15.37292204          nan          nan -15.97780438          nan
          nan -15.45333507          nan          nan -14.70552936
          nan          nan -15.02174332]
  warnings.warn(


CV elapsed time: 7277.04s
best parameter:  {'amnesic': 0.9, 'mu': 1e-06} ; score:  -1.4705529e+01


In [35]:
 import Code.SVDPLS
importlib.reload(Code.SVDPLS)
from Code.SVDPLS import SVDPLS



n_comp_lst=list(range(1,6))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SVDPLS(), parameters,
                   cv=TimeSeriesSplit(n_splits=28, test_size=30),
                   n_jobs=1,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=28, test_size=30),
             estimator=SVDPLS(), n_jobs=1,
             param_grid={'n_components': [1, 2, 3, 4, 5]},
             scoring='neg_root_mean_squared_error')


CV elapsed time: 56.86s
best parameter:  {'n_components': 3} ; score:  -9.5095445e+00
